In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import re

import ast

In [2]:
df = pd.read_csv('data/row_papersdata_v13.csv')
df.head(3)

,_id,title,authors,venue,year,keywords,fos,references,n_citation,page_start,page_end,lang,volume,issue,issn,isbn,doi,pdf,url,abstract
0,53e99784b7602d9701f3e151,A solution to the problem of touching and brok...,"[{'_id': '53f46797dabfaeb22f542630', 'name': '...","{'_id': '53a72a4920f7420be8bfa51b', 'name_d': ...",1993.0,"['handwriting recognition', 'prototypes', 'ima...","['Intelligent character recognition', 'Pattern...","['53e99cf5b7602d97025ace63', '557e8a7a6fee0fe9...",17.0,602,605,en,NaN,NaN,NaN,NaN,10.1109/ICDAR.1993.395663,NaN,['http://dx.doi.org/10.1109/ICDAR.1993.395663'],NaN
1,53e99784b7602d9701f3e161,360°,"[{'_id': '53f46946dabfaec09f24b4ed', 'name': '...","{'_id': '5390a74a20f70186a0e8b40b', 'raw': 'AC...",2009.0,"['global high technology', 'daily short-distan...",NaN,NaN,NaN,39,39,en,NaN,NaN,NaN,NaN,10.1145/1665137.1665166,NaN,NaN,360° represents the concerns that are addresse...
2,53e99784b7602d9701f3f992,Firewalls.,"[{'_id': '53f453aedabfaeb22f4f637d', 'name': '...",{'raw': 'Encyclopedia of Cryptography and Secu...,2011.0,[],NaN,NaN,0.0,474,482,en,NaN,NaN,NaN,NaN,NaN,NaN,['http://dx.doi.org/10.1007/978-1-4419-5906-5_...,NaN


In [3]:
def get_names(series, needed='name') -> list:
    def _parse_row(row):
        if row is not None:
            try: values_dict = ast.literal_eval(row)
            except: return([])
            
            names = [None] * len(values_dict)
            for key, value in enumerate(values_dict):
                try: names[key] = value[needed]
                except: return([])
                
            return(names)
        return([])
    
    parsed_series = [None] * series.shape[0]
    for key, row in enumerate(tqdm(df['authors'].iloc())):
        names = _parse_row(row)
        parsed_series[key] = list(set(names))
    return parsed_series

In [4]:
authors_names = get_names(df['authors'], needed='name')

385524it [00:23, 16118.27it/s]


In [5]:
org_names = get_names(df['authors'], needed='org')

385524it [00:22, 17007.12it/s]


In [6]:
raw_venue_names = get_names(df['venue'], needed='raw')

385524it [00:21, 17684.27it/s]


In [7]:
df['auther_name'] = authors_names
df['org_names'] = org_names

In [8]:
print(f'Before: {df.shape}')
df = df.explode('auther_name')
# df = df.explode('org_names')
print(f'After:  {df.shape}')

Before: (385524, 22)
After:  (1188750, 22)


In [9]:
df.to_csv('data/last_version.csv')